
<p align="center">
    <img src="../assets/imgs/creat_geohash.svg" alt="drawing" style="width:600px;"/>
</p>

In [ ]:
import geopandas as gpd
from rasterio.mask import mask
import rasterio as rio

In [ ]:
clear = gpd.read_file('all_data.gpkg').to_crs(4326)
h7=gpd.read_file(f'amostra7_fulldata.gpkg')

In [ ]:
def get_weith_mpb(row):
    nota = {
          1:5,    # 'Pastagem':'#EDDE8E', 
          2:10,   # 'Formação Florestal':'#32A65E', 
          3:5,    # 'Agricultura':'#E974ED', 
          4:5,    # 'Agropecuária':'#FFFFB2', 
          5:5,    # 'Silvicultura':'#7A5900', 
          6:10,    # 'Áreas Urbanizadas':'#D4271E', 
          7:10,    # 'Formação Savânica':'#7DC975', 
          8:15,    # 'Água':'#0000FF', 
          9:10,    # 'Áreas Alagáveis':'#026975', 
         10:10,     # 'Restinga Arbórea':'#02D659', 
         11:10,     # 'Formação Campestre':'#D6BC74', 
         12:5,     # 'Lavouras Perenes':'#D68FE2', 
         13:5,     # 'Mangue':'#04381D', 
         14:15,     # 'Mineração':'#9C0027', 
         15:10,     # 'Solo exposto':'#FFAA5F', 
         16:5,     # 'Regeneração':'#AD975A', 
         17:5,     # 'Apicum':'#FC8114', 
         18:5,     # 'Restinga Herbácea':'#AD5100', 
         19:5   # 'Asfalto':'#CECECE',
        }
    remap = {
         3: 2,
         4: 7,
         5: 13,
         6: 9,
         49: 10,
         13: 2,
         32: 17,
         11: 9,
         50: 18,
         12: 11,
         29: 15,
         9: 5,
         14: 4,
         15: 1,
         20: 3,
         21: 4,
         39: 3,
         40: 3,
         41: 3,
         46: 12,
         47: 12,
         48: 12,
         62: 3,
         24: 6,
         25: 15,
         30: 13,
         33: 8,
         31: 8,
         26: 8,
         23: 15,
        35:12
    }
    mpb = rio.open('data/brazil_col8.tif')
    out_mpb, out_transform_mpb = mask(mpb, [row['geometry']], all_touched=True, crop=True, nodata=0)
    dados_mpb = out_mpb[out_mpb != mpb.nodata]
    valores_mpb, contagens_mpb = np.unique([i for i in dados_mpb if i > 0], return_counts=True)
    try:
        return sum([nota[remap[i]] for i in valores_mpb])
    except:
        print(valores_mpb)
        raise f'error {valores_mpb}'

def loop(args):
    i, info = args
    tmp = _clear.intersection(info['geometry']).to_crs(5880)
    tmp_info = info.to_dict()
    tmp_info['hash6'] = tmp_info['hash'][0:6]
    tmp_info['weiht'] = get_weith_mpb(tmp_info)
    tmp_info['weiht_geometry'] = int((tmp[tmp.area > 0].area / 1000).sum())

    return tmp_info

def getsum(row):
    if row['weiht_geometry'] > 0:
        return np.log2(row['weiht_geometry']).astype('int') + row['weiht']
    return 0



def get_group_id(x):
    for key, value in cluster_id.items():
        if x in value:
            return key

In [ ]:
_clear = clear.to_crs(4326)
_clear['geometry'] = _clear['geometry'].make_valid()
_h7 = h7.to_crs(4326)
total = len(_h7)

with Pool(16) as p:
    rows = list(tqdm(p.imap(loop, _h7.iterrows()), total=total))

ttd = gpd.GeoDataFrame(rows)
ttd.to_file('area_com_peso.gpkg')

ttd2 = ttd

cluster_id = {
    1:['0','1','2','3'],
    2:['8','9','b','c'],
    3:['4','5','6','7'],
    4:['d','e','f','g'],
    5:['h','j','k','m'],
    6:['s','t','u','v'],
    7:['n','p','q','r'],
    8:['w','x','y','z'],
}



In [ ]:
rows_aprovados = []
for h in tqdm(ttd2['hash6'].unique()):
    tmp = ttd2[ttd2['hash6'] == h].reset_index(drop=True)
    tmp['cid'] = tmp['hash'].str[-1] 
    
    tmp1 = tmp.copy()
    tmp1 = tmp1[tmp1['weiht_geometry'] > 0]
    tmp1['sum'] = np.log2(tmp1['weiht_geometry']).astype('int') + tmp1['weiht']
    tmp1 = tmp1.sort_values('sum', ascending=False)[0:2]
    if len(tmp1) > 0:
        cselect = []
        for ci in cluster_id:
            tmpc = tmp.drop(tmp1.index)
            try:
                tmpc = tmpc[tmpc['cid'].isin(cluster_id[ci])].sort_values('weiht', ascending=False).iloc[0]
                cselect.append(tmpc)
            except:
                ...
        tmp1 = pd.concat([tmp1.set_crs(4326,allow_override=True),gpd.GeoDataFrame(cselect,crs=4326)])
        tmp2 = tmp.drop(tmp1.index)
        tmp2 = tmp2[tmp2['weiht'] > 0]
        tmp2 = tmp2.sort_values('weiht', ascending=False)[0:(10-len(tmp1))]
        tmp3 = pd.concat([tmp1,tmp2.set_crs(4326,allow_override=True)])
        rows_aprovados.append(tmp3)


gdf_br_amostras = pd.concat(rows_aprovados)

In [ ]:
gdf_br_amostras.to_crs(3857).to_file('amostra_rois_3857.gpkg')